In [1]:
from scrap import Scraper

In [2]:
url = "https://www.geeksforgeeks.org/queue-data-structure/"

In [3]:
scrp = Scraper(url)

In [4]:
string, text = scrp.scrape_website()

In [5]:
len(string)

34612